# リアルタイム音響処理とモニタリング

このノートブックでは、リアルタイムでの音響処理とモニタリングのシミュレーションを学習します。
実際のマイク入力の代わりに、生成した音響データを使用してリアルタイム処理の概念を学びます。

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from collections import deque
import time
from scipy import signal
from scipy.fft import fft, fftfreq
import threading
import queue
from pydub import AudioSegment
from pydub.generators import Sine, Square, Sawtooth
import warnings
warnings.filterwarnings('ignore')

# 日本語フォントの設定
plt.rcParams['font.family'] = 'DejaVu Sans'

# Jupyter notebook内でアニメーションを表示
%matplotlib notebook

## 1. リアルタイムデータストリームのシミュレーション

In [ ]:
class AudioStreamSimulator:
    """
    音響ストリームをシミュレートするクラス
    """
    def __init__(self, sample_rate=1000, chunk_size=256):
        self.sample_rate = sample_rate
        self.chunk_size = chunk_size
        self.time_counter = 0
        self.is_running = False
        self.data_queue = queue.Queue()
        
    def generate_audio_chunk(self):
        """
        音響チャンクを生成
        """
        t_start = self.time_counter / self.sample_rate
        t_end = (self.time_counter + self.chunk_size) / self.sample_rate
        t = np.linspace(t_start, t_end, self.chunk_size, endpoint=False)
        
        # 複数の音響成分を組み合わせ
        # 基本周波数（時間とともに変化）
        freq1 = 200 + 100 * np.sin(2 * np.pi * 0.1 * t_start)
        signal1 = np.sin(2 * np.pi * freq1 * t)
        
        # 高調波
        freq2 = 300 + 50 * np.cos(2 * np.pi * 0.05 * t_start)
        signal2 = 0.5 * np.sin(2 * np.pi * freq2 * t)
        
        # ノイズ
        noise = 0.1 * np.random.randn(self.chunk_size)
        
        # 不定期なパルス
        if np.random.random() < 0.1:  # 10%の確率でパルス
            pulse_start = np.random.randint(0, self.chunk_size - 50)
            pulse = np.zeros(self.chunk_size)
            pulse[pulse_start:pulse_start+50] = 2 * np.sin(2 * np.pi * 500 * 
                                                          t[pulse_start:pulse_start+50])
        else:
            pulse = np.zeros(self.chunk_size)
        
        # 全ての成分を合成
        combined_signal = signal1 + signal2 + noise + pulse
        
        self.time_counter += self.chunk_size
        return combined_signal, t
    
    def start_stream(self):
        """
        ストリーミング開始
        """
        self.is_running = True
        
        def stream_thread():
            while self.is_running:
                chunk, time_chunk = self.generate_audio_chunk()
                self.data_queue.put((chunk, time_chunk))
                time.sleep(self.chunk_size / self.sample_rate)  # リアルタイムシミュレーション
        
        self.thread = threading.Thread(target=stream_thread)
        self.thread.start()
    
    def stop_stream(self):
        """
        ストリーミング停止
        """
        self.is_running = False
        if hasattr(self, 'thread'):
            self.thread.join()
    
    def get_chunk(self):
        """
        次のチャンクを取得
        """
        try:
            return self.data_queue.get_nowait()
        except queue.Empty:
            return None, None

# シミュレータのテスト
simulator = AudioStreamSimulator()
chunk, t = simulator.generate_audio_chunk()

plt.figure(figsize=(12, 4))
plt.plot(t, chunk)
plt.title('Sample Audio Chunk')
plt.xlabel('Time (s)')
plt.ylabel('Amplitude')
plt.grid(True)
plt.show()

print(f"Chunk size: {len(chunk)} samples")
print(f"Duration: {len(chunk)/simulator.sample_rate:.3f} seconds")

## 2. リアルタイム波形モニタリング

In [ ]:
class RealTimeWaveformMonitor:
    """
    リアルタイム波形モニタ
    """
    def __init__(self, buffer_size=2000, sample_rate=1000):
        self.buffer_size = buffer_size
        self.sample_rate = sample_rate
        self.audio_buffer = deque(maxlen=buffer_size)
        self.time_buffer = deque(maxlen=buffer_size)
        
        # 初期化
        for _ in range(buffer_size):
            self.audio_buffer.append(0)
            self.time_buffer.append(0)
    
    def update_buffer(self, new_data, new_time):
        """
        バッファを新しいデータで更新
        """
        for sample, t in zip(new_data, new_time):
            self.audio_buffer.append(sample)
            self.time_buffer.append(t)
    
    def get_display_data(self):
        """
        表示用データを取得
        """
        return np.array(self.time_buffer), np.array(self.audio_buffer)

# モニタの作成とテスト
monitor = RealTimeWaveformMonitor()

# 複数チャンクでテスト
simulator = AudioStreamSimulator()
for i in range(10):
    chunk, t_chunk = simulator.generate_audio_chunk()
    monitor.update_buffer(chunk, t_chunk)

time_data, audio_data = monitor.get_display_data()

plt.figure(figsize=(15, 6))
plt.plot(time_data, audio_data)
plt.title('Real-time Waveform Buffer')
plt.xlabel('Time (s)')
plt.ylabel('Amplitude')
plt.grid(True)
plt.show()

print(f"Buffer contains {len(audio_data)} samples")
print(f"Time range: {time_data[0]:.3f} - {time_data[-1]:.3f} seconds")

## 3. リアルタイムスペクトラム解析

In [ ]:
class RealTimeSpectrumAnalyzer:
    """
    リアルタイムスペクトラム解析器
    """
    def __init__(self, fft_size=512, sample_rate=1000, overlap=0.5):
        self.fft_size = fft_size
        self.sample_rate = sample_rate
        self.overlap = overlap
        self.hop_size = int(fft_size * (1 - overlap))
        
        # 窓関数
        self.window = signal.windows.hann(fft_size)
        
        # 周波数軸
        self.freqs = fftfreq(fft_size, 1/sample_rate)[:fft_size//2]
        
        # スペクトログラム用バッファ
        self.spectrogram_buffer = deque(maxlen=100)  # 100フレーム分
        
        # 音響特徴量の履歴
        self.spectral_centroid_history = deque(maxlen=200)
        self.spectral_rolloff_history = deque(maxlen=200)
        self.rms_history = deque(maxlen=200)
        
    def analyze_chunk(self, audio_chunk):
        """
        音響チャンクを解析
        """
        if len(audio_chunk) < self.fft_size:
            # ゼロパディング
            padded_chunk = np.zeros(self.fft_size)
            padded_chunk[:len(audio_chunk)] = audio_chunk
            audio_chunk = padded_chunk
        
        # 窓関数の適用
        windowed_signal = audio_chunk[:self.fft_size] * self.window
        
        # FFT
        fft_result = fft(windowed_signal)
        magnitude_spectrum = np.abs(fft_result[:self.fft_size//2])
        power_spectrum = magnitude_spectrum ** 2
        
        # スペクトログラムバッファに追加
        self.spectrogram_buffer.append(magnitude_spectrum)
        
        # 音響特徴量の計算
        # スペクトラル重心
        if np.sum(power_spectrum) > 0:
            spectral_centroid = np.sum(self.freqs * power_spectrum) / np.sum(power_spectrum)
        else:
            spectral_centroid = 0
        
        # スペクトラルロールオフ（85%エネルギーポイント）
        cumsum_power = np.cumsum(power_spectrum)
        if cumsum_power[-1] > 0:
            rolloff_idx = np.where(cumsum_power >= 0.85 * cumsum_power[-1])[0]
            spectral_rolloff = self.freqs[rolloff_idx[0]] if len(rolloff_idx) > 0 else 0
        else:
            spectral_rolloff = 0
        
        # RMS
        rms = np.sqrt(np.mean(audio_chunk ** 2))
        
        # 履歴に追加
        self.spectral_centroid_history.append(spectral_centroid)
        self.spectral_rolloff_history.append(spectral_rolloff)
        self.rms_history.append(rms)
        
        return {
            'magnitude_spectrum': magnitude_spectrum,
            'power_spectrum': power_spectrum,
            'spectral_centroid': spectral_centroid,
            'spectral_rolloff': spectral_rolloff,
            'rms': rms
        }
    
    def get_spectrogram(self):
        """
        スペクトログラムデータを取得
        """
        if len(self.spectrogram_buffer) == 0:
            return np.zeros((len(self.freqs), 1))
        return np.array(list(self.spectrogram_buffer)).T
    
    def get_feature_history(self):
        """
        特徴量履歴を取得
        """
        return {
            'spectral_centroid': np.array(self.spectral_centroid_history),
            'spectral_rolloff': np.array(self.spectral_rolloff_history),
            'rms': np.array(self.rms_history)
        }

# スペクトラム解析器のテスト
analyzer = RealTimeSpectrumAnalyzer()
simulator = AudioStreamSimulator()

# 複数チャンクの解析
results = []
for i in range(20):
    chunk, _ = simulator.generate_audio_chunk()
    result = analyzer.analyze_chunk(chunk)
    results.append(result)

# 結果の可視化
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# 最新のスペクトラム
axes[0, 0].semilogy(analyzer.freqs, results[-1]['magnitude_spectrum'])
axes[0, 0].set_title('Latest Magnitude Spectrum')
axes[0, 0].set_xlabel('Frequency (Hz)')
axes[0, 0].set_ylabel('Magnitude')
axes[0, 0].grid(True)

# スペクトログラム
spectrogram = analyzer.get_spectrogram()
im = axes[0, 1].imshow(spectrogram, aspect='auto', origin='lower', 
                       extent=[0, len(results), 0, analyzer.freqs[-1]], cmap='hot')
axes[0, 1].set_title('Spectrogram')
axes[0, 1].set_xlabel('Time Frame')
axes[0, 1].set_ylabel('Frequency (Hz)')
plt.colorbar(im, ax=axes[0, 1], label='Magnitude')

# 特徴量の時間変化
features = analyzer.get_feature_history()
time_frames = np.arange(len(features['spectral_centroid']))

axes[1, 0].plot(time_frames, features['spectral_centroid'], label='Spectral Centroid')
axes[1, 0].plot(time_frames, features['spectral_rolloff'], label='Spectral Rolloff')
axes[1, 0].set_title('Spectral Features Over Time')
axes[1, 0].set_xlabel('Time Frame')
axes[1, 0].set_ylabel('Frequency (Hz)')
axes[1, 0].legend()
axes[1, 0].grid(True)

# RMS履歴
axes[1, 1].plot(time_frames, features['rms'])
axes[1, 1].set_title('RMS Level Over Time')
axes[1, 1].set_xlabel('Time Frame')
axes[1, 1].set_ylabel('RMS')
axes[1, 1].grid(True)

plt.tight_layout()
plt.show()

print(f"Analyzed {len(results)} audio chunks")
print(f"Latest spectral centroid: {results[-1]['spectral_centroid']:.2f} Hz")
print(f"Latest RMS level: {results[-1]['rms']:.4f}")

## 4. リアルタイム音響イベント検出

In [ ]:
class AudioEventDetector:
    """
    音響イベント検出器
    """
    def __init__(self, sample_rate=1000):
        self.sample_rate = sample_rate
        
        # 閾値設定
        self.onset_threshold = 0.3  # オンセット検出閾値
        self.silence_threshold = 0.01  # 無音検出閾値
        self.frequency_peak_threshold = 0.1  # 周波数ピーク検出閾値
        
        # 履歴バッファ
        self.rms_history = deque(maxlen=10)
        self.spectral_flux_history = deque(maxlen=10)
        self.previous_spectrum = None
        
        # イベント履歴
        self.events = []
        self.frame_count = 0
        
    def detect_events(self, audio_chunk, spectrum_result):
        """
        音響イベントを検出
        """
        events_detected = []
        current_time = self.frame_count * len(audio_chunk) / self.sample_rate
        
        # RMS計算
        rms = np.sqrt(np.mean(audio_chunk ** 2))
        self.rms_history.append(rms)
        
        # 1. オンセット検出（RMSの急激な増加）
        if len(self.rms_history) >= 3:
            rms_diff = rms - np.mean(list(self.rms_history)[:-1])
            if rms_diff > self.onset_threshold:
                events_detected.append({
                    'type': 'onset',
                    'time': current_time,
                    'value': rms_diff,
                    'description': f'Audio onset detected (RMS increase: {rms_diff:.3f})'
                })
        
        # 2. 無音検出
        if rms < self.silence_threshold:
            events_detected.append({
                'type': 'silence',
                'time': current_time,
                'value': rms,
                'description': f'Silence detected (RMS: {rms:.4f})'
            })
        
        # 3. スペクトラルフラックス（周波数内容の変化）
        current_spectrum = spectrum_result['magnitude_spectrum']
        if self.previous_spectrum is not None:
            spectral_flux = np.sum(np.maximum(0, current_spectrum - self.previous_spectrum))
            self.spectral_flux_history.append(spectral_flux)
            
            # 高いスペクトラルフラックスは新しい音の出現を示す
            if len(self.spectral_flux_history) >= 3:
                flux_threshold = np.mean(list(self.spectral_flux_history)[:-1]) + 2 * np.std(list(self.spectral_flux_history)[:-1])
                if spectral_flux > flux_threshold and spectral_flux > 10:
                    events_detected.append({
                        'type': 'spectral_change',
                        'time': current_time,
                        'value': spectral_flux,
                        'description': f'Spectral change detected (flux: {spectral_flux:.2f})'
                    })
        
        # 4. 周波数ピーク検出
        spectrum = spectrum_result['magnitude_spectrum']
        peaks, properties = signal.find_peaks(spectrum, height=np.max(spectrum) * self.frequency_peak_threshold)
        
        if len(peaks) > 3:  # 多くのピークがある場合
            # 周波数を計算
            freqs = np.fft.fftfreq(len(spectrum) * 2, 1/self.sample_rate)[:len(spectrum)]
            peak_freqs = freqs[peaks]
            
            events_detected.append({
                'type': 'harmonic_content',
                'time': current_time,
                'value': len(peaks),
                'description': f'Harmonic content detected ({len(peaks)} peaks at {peak_freqs[:3]:.0f} Hz)'
            })
        
        # 5. 高エネルギー検出
        if rms > 1.0:  # 高いRMSレベル
            events_detected.append({
                'type': 'high_energy',
                'time': current_time,
                'value': rms,
                'description': f'High energy event (RMS: {rms:.3f})'
            })
        
        # イベントを履歴に追加
        self.events.extend(events_detected)
        
        # 前のスペクトラムを保存
        self.previous_spectrum = current_spectrum.copy()
        self.frame_count += 1
        
        return events_detected
    
    def get_event_summary(self):
        """
        イベントサマリーを取得
        """
        if not self.events:
            return "No events detected"
        
        event_types = {}
        for event in self.events:
            event_type = event['type']
            if event_type not in event_types:
                event_types[event_type] = 0
            event_types[event_type] += 1
        
        return event_types

# イベント検出器のテスト
detector = AudioEventDetector()
analyzer = RealTimeSpectrumAnalyzer()
simulator = AudioStreamSimulator()

print("Processing audio stream and detecting events...")
print("-" * 50)

all_events = []
for i in range(30):
    chunk, _ = simulator.generate_audio_chunk()
    spectrum_result = analyzer.analyze_chunk(chunk)
    events = detector.detect_events(chunk, spectrum_result)
    
    # イベントが検出された場合に表示
    for event in events:
        print(f"[{event['time']:.2f}s] {event['type'].upper()}: {event['description']}")
    
    all_events.extend(events)

print("-" * 50)
print("Event Summary:")
summary = detector.get_event_summary()
for event_type, count in summary.items():
    print(f"  {event_type}: {count} occurrences")

print(f"\nTotal events detected: {len(all_events)}")
print(f"Analysis duration: {detector.frame_count * 256 / 1000:.2f} seconds")

## 5. リアルタイム音響効果の適用

In [ ]:
class RealTimeAudioProcessor:
    """
    リアルタイム音響処理器
    """
    def __init__(self, sample_rate=1000):
        self.sample_rate = sample_rate
        
        # エフェクトパラメータ
        self.effects = {
            'gain': 1.0,
            'lowpass_freq': 500,
            'highpass_freq': 50,
            'echo_delay': 0.5,
            'echo_decay': 0.3,
            'compressor_threshold': 0.7,
            'compressor_ratio': 4.0
        }
        
        # フィルタ係数
        self.update_filters()
        
        # エコー用ディレイバッファ
        delay_samples = int(self.effects['echo_delay'] * sample_rate)
        self.delay_buffer = deque(maxlen=delay_samples, iterable=[0] * delay_samples)
        
        # コンプレッサ用状態
        self.compressor_state = 0
        
    def update_filters(self):
        """
        フィルタ係数を更新
        """
        # ローパスフィルタ
        nyquist = self.sample_rate / 2
        low_freq = min(self.effects['lowpass_freq'], nyquist - 1)
        self.lowpass_b, self.lowpass_a = signal.butter(4, low_freq / nyquist, btype='low')
        
        # ハイパスフィルタ
        high_freq = max(self.effects['highpass_freq'], 1)
        self.highpass_b, self.highpass_a = signal.butter(4, high_freq / nyquist, btype='high')
    
    def apply_gain(self, audio_chunk):
        """
        ゲインを適用
        """
        return audio_chunk * self.effects['gain']
    
    def apply_filters(self, audio_chunk):
        """
        フィルタを適用
        """
        # ローパス
        filtered = signal.lfilter(self.lowpass_b, self.lowpass_a, audio_chunk)
        # ハイパス
        filtered = signal.lfilter(self.highpass_b, self.highpass_a, filtered)
        return filtered
    
    def apply_echo(self, audio_chunk):
        """
        エコーエフェクトを適用
        """
        output = np.zeros_like(audio_chunk)
        
        for i, sample in enumerate(audio_chunk):
            # ディレイバッファから古いサンプルを取得
            delayed_sample = self.delay_buffer[0]
            
            # エコーを加算
            output[i] = sample + delayed_sample * self.effects['echo_decay']
            
            # 新しいサンプルをディレイバッファに追加
            self.delay_buffer.append(sample)
        
        return output
    
    def apply_compressor(self, audio_chunk):
        """
        シンプルなコンプレッサを適用
        """
        threshold = self.effects['compressor_threshold']
        ratio = self.effects['compressor_ratio']
        
        output = np.zeros_like(audio_chunk)
        
        for i, sample in enumerate(audio_chunk):
            abs_sample = abs(sample)
            
            if abs_sample > threshold:
                # 圧縮を適用
                excess = abs_sample - threshold
                compressed_excess = excess / ratio
                compressed_amplitude = threshold + compressed_excess
                output[i] = np.sign(sample) * compressed_amplitude
            else:
                output[i] = sample
        
        return output
    
    def process_chunk(self, audio_chunk, effects_enabled=None):
        """
        音響チャンクを処理
        """
        if effects_enabled is None:
            effects_enabled = ['gain', 'filters', 'echo', 'compressor']
        
        processed = audio_chunk.copy()
        
        # エフェクトチェーンの適用
        if 'gain' in effects_enabled:
            processed = self.apply_gain(processed)
        
        if 'filters' in effects_enabled:
            processed = self.apply_filters(processed)
        
        if 'echo' in effects_enabled:
            processed = self.apply_echo(processed)
        
        if 'compressor' in effects_enabled:
            processed = self.apply_compressor(processed)
        
        return processed
    
    def set_effect_parameter(self, effect_name, value):
        """
        エフェクトパラメータを設定
        """
        if effect_name in self.effects:
            self.effects[effect_name] = value
            
            # フィルタパラメータが変更された場合は再計算
            if effect_name in ['lowpass_freq', 'highpass_freq']:
                self.update_filters()

# リアルタイム処理のデモ
processor = RealTimeAudioProcessor()
simulator = AudioStreamSimulator()

# 異なるエフェクト設定でテスト
effect_configs = [
    {'name': 'Original', 'effects': []},
    {'name': 'Gain Only', 'effects': ['gain']},
    {'name': 'Filtered', 'effects': ['gain', 'filters']},
    {'name': 'With Echo', 'effects': ['gain', 'filters', 'echo']},
    {'name': 'Full Processing', 'effects': ['gain', 'filters', 'echo', 'compressor']}
]

# パラメータ調整
processor.set_effect_parameter('gain', 1.5)
processor.set_effect_parameter('lowpass_freq', 300)
processor.set_effect_parameter('echo_decay', 0.4)

# 各設定でのテスト
fig, axes = plt.subplots(len(effect_configs), 1, figsize=(15, 2*len(effect_configs)))

# テスト用音響データ生成
test_chunk, test_time = simulator.generate_audio_chunk()

for i, config in enumerate(effect_configs):
    if config['name'] == 'Original':
        processed_chunk = test_chunk
    else:
        processed_chunk = processor.process_chunk(test_chunk, config['effects'])
    
    axes[i].plot(test_time, processed_chunk)
    axes[i].set_title(f'{config["name"]} (Effects: {config["effects"]})')
    axes[i].set_ylabel('Amplitude')
    axes[i].grid(True)
    
    # RMS計算
    rms = np.sqrt(np.mean(processed_chunk**2))
    axes[i].text(0.02, 0.95, f'RMS: {rms:.3f}', transform=axes[i].transAxes, 
                bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))

axes[-1].set_xlabel('Time (s)')
plt.tight_layout()
plt.show()

# エフェクトパラメータの表示
print("Current Effect Parameters:")
for param, value in processor.effects.items():
    print(f"  {param}: {value}")

## 6. 統合リアルタイムシステムのデモ

In [ ]:
class IntegratedAudioSystem:
    """
    統合音響処理システム
    """
    def __init__(self, sample_rate=1000, chunk_size=256):
        self.sample_rate = sample_rate
        self.chunk_size = chunk_size
        
        # 各コンポーネントの初期化
        self.simulator = AudioStreamSimulator(sample_rate, chunk_size)
        self.monitor = RealTimeWaveformMonitor(buffer_size=1000, sample_rate=sample_rate)
        self.analyzer = RealTimeSpectrumAnalyzer(fft_size=512, sample_rate=sample_rate)
        self.detector = AudioEventDetector(sample_rate=sample_rate)
        self.processor = RealTimeAudioProcessor(sample_rate=sample_rate)
        
        # システム状態
        self.processing_stats = {
            'chunks_processed': 0,
            'total_events': 0,
            'processing_time': 0,
            'avg_rms': 0
        }
    
    def process_single_frame(self, enable_effects=True):
        """
        単一フレームを処理
        """
        start_time = time.time()
        
        # 1. 音響データ取得
        chunk, time_chunk = self.simulator.generate_audio_chunk()
        
        # 2. エフェクト処理
        if enable_effects:
            processed_chunk = self.processor.process_chunk(chunk)
        else:
            processed_chunk = chunk
        
        # 3. モニタリング
        self.monitor.update_buffer(processed_chunk, time_chunk)
        
        # 4. スペクトラム解析
        spectrum_result = self.analyzer.analyze_chunk(processed_chunk)
        
        # 5. イベント検出
        events = self.detector.detect_events(processed_chunk, spectrum_result)
        
        # 6. 統計更新
        processing_time = time.time() - start_time
        self.processing_stats['chunks_processed'] += 1
        self.processing_stats['total_events'] += len(events)
        self.processing_stats['processing_time'] += processing_time
        self.processing_stats['avg_rms'] = (
            (self.processing_stats['avg_rms'] * (self.processing_stats['chunks_processed'] - 1) + 
             spectrum_result['rms']) / self.processing_stats['chunks_processed']
        )
        
        return {
            'original_chunk': chunk,
            'processed_chunk': processed_chunk,
            'time_chunk': time_chunk,
            'spectrum_result': spectrum_result,
            'events': events,
            'processing_time': processing_time
        }
    
    def run_demo(self, num_frames=50, enable_effects=True):
        """
        デモンストレーション実行
        """
        print(f"Running integrated audio system demo for {num_frames} frames...")
        print(f"Effects enabled: {enable_effects}")
        print("-" * 60)
        
        results = []
        
        for frame in range(num_frames):
            result = self.process_single_frame(enable_effects)
            results.append(result)
            
            # 重要なイベントを表示
            for event in result['events']:
                if event['type'] in ['onset', 'high_energy', 'spectral_change']:
                    print(f"[Frame {frame:2d}] {event['description']}")
            
            # 進行状況表示
            if (frame + 1) % 10 == 0:
                print(f"Processed {frame + 1}/{num_frames} frames...")
        
        return results
    
    def get_system_report(self):
        """
        システムレポートを生成
        """
        stats = self.processing_stats
        if stats['chunks_processed'] == 0:
            return "No data processed yet."
        
        avg_processing_time = stats['processing_time'] / stats['chunks_processed']
        total_duration = stats['chunks_processed'] * self.chunk_size / self.sample_rate
        event_rate = stats['total_events'] / total_duration if total_duration > 0 else 0
        
        report = f"""
System Performance Report:
  Total chunks processed: {stats['chunks_processed']}
  Total processing time: {stats['processing_time']:.3f} seconds
  Average processing time per chunk: {avg_processing_time*1000:.2f} ms
  Audio duration processed: {total_duration:.2f} seconds
  Real-time performance: {(total_duration/stats['processing_time']):.1f}x
  
Audio Analysis:
  Total events detected: {stats['total_events']}
  Event detection rate: {event_rate:.2f} events/second
  Average RMS level: {stats['avg_rms']:.4f}
  
System Status: {'✓ Real-time capable' if avg_processing_time < (self.chunk_size/self.sample_rate) else '⚠ Not real-time capable'}
"""
        return report

# 統合システムのデモ
system = IntegratedAudioSystem()

# システム設定
system.processor.set_effect_parameter('gain', 1.2)
system.processor.set_effect_parameter('lowpass_freq', 400)
system.processor.set_effect_parameter('echo_decay', 0.3)

# デモ実行
demo_results = system.run_demo(num_frames=30, enable_effects=True)

print("\n" + "="*60)
print(system.get_system_report())

# 結果の可視化
fig, axes = plt.subplots(3, 1, figsize=(15, 12))

# 波形モニタリング
time_data, audio_data = system.monitor.get_display_data()
axes[0].plot(time_data, audio_data)
axes[0].set_title('Real-time Waveform Monitor (Last 1000 samples)')
axes[0].set_ylabel('Amplitude')
axes[0].grid(True)

# スペクトログラム
spectrogram = system.analyzer.get_spectrogram()
if spectrogram.shape[1] > 1:
    im = axes[1].imshow(spectrogram, aspect='auto', origin='lower', 
                       extent=[0, spectrogram.shape[1], 0, system.analyzer.freqs[-1]], cmap='hot')
    axes[1].set_title('Real-time Spectrogram')
    axes[1].set_ylabel('Frequency (Hz)')
    plt.colorbar(im, ax=axes[1], label='Magnitude')

# 特徴量履歴
features = system.analyzer.get_feature_history()
if len(features['spectral_centroid']) > 0:
    frame_indices = np.arange(len(features['spectral_centroid']))
    axes[2].plot(frame_indices, features['spectral_centroid'], label='Spectral Centroid')
    axes[2].plot(frame_indices, features['rms'] * 1000, label='RMS x1000')  # スケール調整
    axes[2].set_title('Audio Features Over Time')
    axes[2].set_xlabel('Frame Index')
    axes[2].set_ylabel('Value')
    axes[2].legend()
    axes[2].grid(True)

plt.tight_layout()
plt.show()

# イベント検出サマリー
event_summary = system.detector.get_event_summary()
print("\nEvent Detection Summary:")
for event_type, count in event_summary.items():
    print(f"  {event_type}: {count} occurrences")